In [38]:
# from datasets import load_dataset
# 
# datasets = load_dataset("squad", split="train[:5000]")

from datasets import load_dataset

# use name="sample-10BT" to use the 10BT sample
datasets = load_dataset(
    "HuggingFaceFW/fineweb-edu", 
    name="sample-10BT", 
    # split="train[:5000]", 
    # streaming=True,
)

'''
IterableDatasetDict({
    train: IterableDataset({
        features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],
        n_shards: 14
    })
})
'''


Resolving data files:   0%|          | 0/1630 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/98 [00:00<?, ?it/s]

"\nIterableDatasetDict({\n    train: IterableDataset({\n        features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],\n        n_shards: 14\n    })\n})\n"

In [39]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],
        num_rows: 9672101
    })
})


In [40]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(data, num_examples=10):
    assert num_examples <= len(data), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(data)-1)
        while pick in picks:
            pick = random.randint(0, len(data)-1)
        picks.append(pick)
    
    df = pd.DataFrame(data[picks])
    for column, typ in data.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [41]:
show_random_elements(datasets['train'])

In [42]:
from transformers import AutoTokenizer
from functools import partial

model_checkpoint = "Qwen/Qwen2-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_function(examples, tokenizer):
    return tokenizer(examples["text"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenized_datasets = datasets.map(partial(tokenize_function, tokenizer=tokenizer), batched=True, num_proc=12, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/9672101 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets["train"][1]

In [ ]:
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [33]:
import torch
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
)

train_dataloader = torch.utils.data.DataLoader(
    lm_datasets["train"], 
    shuffle=True, 
    batch_size=8, 
    collate_fn=data_collator
)

In [1]:
dataset = dataset.train_test_split(test_size=0.2)

UnicodeEncodeError: 'gbk' codec can't encode character '\u2022' in position 2417: illegal multibyte sequence

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    cache_dir='/Volumes/Aurora/.cache/huggingface/hub',
)

def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)), end='\n\n')

prompt = """### User: Hello\n\n### Assistant: Hi, how can I help you?"""
print_tokens_with_ids(prompt)  # [..., ('▁Hello', 15043), ('<0x0A>', 13), ('<0x0A>', 13), ('##', 2277), ('#', 29937), ('▁Ass', 4007), ('istant', 22137), (':', 29901), ...]

response_template = "### Assistant:"
print_tokens_with_ids(response_template)  # [('▁###', 835), ('▁Ass', 4007), ('istant', 22137), (':', 29901)]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[('###', 14711), ('ĠUser', 2724), (':', 25), ('ĠHello', 22691), ('ĊĊ', 271), ('###', 14711), ('ĠAssistant', 22103), (':', 25), ('ĠHi', 21694), (',', 11), ('Ġhow', 1268), ('Ġcan', 649), ('ĠI', 358), ('Ġhelp', 1520), ('Ġyou', 499), ('?', 30)]

[('###', 14711), ('ĠAssistant', 22103), (':', 25)]



In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    cache_dir='/Volumes/Aurora/.cache/huggingface/hub',
)

def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)), end='\n\n')

prompt = """### User: Hello\n\n### Assistant: Hi, how can I help you?"""
print_tokens_with_ids(prompt)  # [..., ('▁Hello', 15043), ('<0x0A>', 13), ('<0x0A>', 13), ('##', 2277), ('#', 29937), ('▁Ass', 4007), ('istant', 22137), (':', 29901), ...]

response_template = "### Assistant:"
print_tokens_with_ids(response_template)  # [('▁###', 835), ('▁Ass', 4007), ('istant', 22137), (':', 29901)]

modified_response_template = "\n### Assistant:"
print_tokens_with_ids(modified_response_template)  # [('▁###', 835), ('▁Ass', 4007), ('istant', 22137), (':', 29901)]

[('▁###', 835), ('▁User', 4911), (':', 29901), ('▁Hello', 15043), ('<0x0A>', 13), ('<0x0A>', 13), ('##', 2277), ('#', 29937), ('▁Ass', 4007), ('istant', 22137), (':', 29901), ('▁Hi', 6324), (',', 29892), ('▁how', 920), ('▁can', 508), ('▁I', 306), ('▁help', 1371), ('▁you', 366), ('?', 29973)]

[('▁###', 835), ('▁Ass', 4007), ('istant', 22137), (':', 29901)]

[('▁', 29871), ('<0x0A>', 13), ('##', 2277), ('#', 29937), ('▁Ass', 4007), ('istant', 22137), (':', 29901)]

